In [1]:
from bs4 import BeautifulSoup
import os
import re
import shutil

In [6]:
def get_category_soup(cat):
    root_dir = f'./data/{cat}/www.diablo2.io'
    soup = BeautifulSoup(open(f'{root_dir}/{cat}/index.html'), 'html.parser')
    return root_dir, soup

In [73]:
def parse(cat):
    root_dir, soup = get_category_soup(cat)
    items = soup.find_all('article', class_="element-item")
    items = [{
        'caption': re.sub(r'(discord\nMissing v0.3 Data)', '', \
            "\n".join([line.strip() for line in items[0].text.split("\n") if line.strip() != ""][:-7])).strip(),
        'image': root_dir+item.find('div', {'data-background-image': True})['data-background-image'],
    } for item in items]
    return items

def write_to_dir(items):
    # Build a new folder with all images in the same folder and .txt file pair containing their metadata
    new_dir = f'./data/clean'
    try:
        os.rmdir(new_dir)
    except:
        pass
    os.makedirs(new_dir, exist_ok=True)
    for e in items:
        new_path = f'{new_dir}/{e["image"].split("/")[-1]}'
        shutil.copyfile(e['image'], f'{new_path}')
        image_name = e["image"].split("/")[-1]
        caption_path = f'{new_dir}/{image_name.split(".")[0]}.txt'
        with open(caption_path, 'w') as f:
            f.write(e['caption'])

try: 
    os.rmdir('./data/clean')
except:
    pass
for e in os.listdir('./data'):
    if 'clean' in e: continue
    write_to_dir(parse(e))


In [76]:
!zip -r -q data.zip data/clean